# Compute climate index DD ('drydays'). The computation in the back is taken from the python module xclim

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings

warnings.filterwarnings("ignore")

In [3]:
import glob
from datetime import date

import xarray as xr

In [4]:
from pyhomogenize import open_xrdataset

import index_calculator as xcalc

Select one CORDEX `tas` file and open it with xarray.

In [5]:
path = "/work/ch0636/eddy/pool/obs/MSWEP_V280/DATA"

In [6]:
start = date(1997, 1, 1)
end = date(1998, 1, 1)

yearList = [year for year in range(start.year, end.year)]  # + 1)]

pr = []

for year in yearList:
    pr += glob.glob(path + "/1987/" + "*19870*.nc")

pr

['/work/ch0636/eddy/pool/obs/MSWEP_V280/DATA/1987/1987090.21.nc',
 '/work/ch0636/eddy/pool/obs/MSWEP_V280/DATA/1987/1987084.03.nc',
 '/work/ch0636/eddy/pool/obs/MSWEP_V280/DATA/1987/1987018.21.nc',
 '/work/ch0636/eddy/pool/obs/MSWEP_V280/DATA/1987/1987045.15.nc',
 '/work/ch0636/eddy/pool/obs/MSWEP_V280/DATA/1987/1987094.21.nc',
 '/work/ch0636/eddy/pool/obs/MSWEP_V280/DATA/1987/1987039.21.nc',
 '/work/ch0636/eddy/pool/obs/MSWEP_V280/DATA/1987/1987036.12.nc',
 '/work/ch0636/eddy/pool/obs/MSWEP_V280/DATA/1987/1987081.03.nc',
 '/work/ch0636/eddy/pool/obs/MSWEP_V280/DATA/1987/1987042.12.nc',
 '/work/ch0636/eddy/pool/obs/MSWEP_V280/DATA/1987/1987091.06.nc',
 '/work/ch0636/eddy/pool/obs/MSWEP_V280/DATA/1987/1987064.18.nc',
 '/work/ch0636/eddy/pool/obs/MSWEP_V280/DATA/1987/1987039.06.nc',
 '/work/ch0636/eddy/pool/obs/MSWEP_V280/DATA/1987/1987094.18.nc',
 '/work/ch0636/eddy/pool/obs/MSWEP_V280/DATA/1987/1987035.09.nc',
 '/work/ch0636/eddy/pool/obs/MSWEP_V280/DATA/1987/1987037.15.nc',
 '/work/ch

In [7]:
ds = open_xrdataset(pr)
ds["precipitation"] = ds["precipitation"] / 3
ds["precipitation"].attrs["units"] = "mm h-1"
ds

<xarray.Dataset>
Dimensions:        (lon: 3600, lat: 1800, time: 792)
Coordinates:
  * lon            (lon) float32 -179.9 -179.8 -179.8 ... 179.8 179.9 179.9
  * lat            (lat) float32 89.95 89.85 89.75 ... -89.75 -89.85 -89.95
  * time           (time) object 1987-01-01 00:00:00 ... 1987-04-09 21:00:00
Data variables:
    precipitation  (time, lat, lon) float32 dask.array<chunksize=(1, 1800, 3600), meta=np.ndarray>
Attributes:
    history:       Created on 2021-12-27 11:47
    CF_variables:  ['precipitation']

Now do a preprocessing of the dataset and create an index_calculator.PreProcessing object.
Preprocessor tasks are:
* select a user-given time range (parameter: time_range)
* check time axis for missing, redundant and duplicated time steps
* crop time axis to required left and right border
* write available and requested time range to object

In [8]:
preproc = xcalc.preprocessing(ds, project="MSWEP", crop_time_axis=False)
# preproc = xcalc.preprocessing(ds, crop_time_axis=False)
preproc.preproc

<xarray.Dataset>
Dimensions:  (lon: 3600, lat: 1800, time: 99)
Coordinates:
  * lon      (lon) float32 -179.9 -179.8 -179.8 -179.6 ... 179.8 179.9 179.9
  * lat      (lat) float32 89.95 89.85 89.75 89.65 ... -89.75 -89.85 -89.95
  * time     (time) object 1987-01-01 00:00:00 ... 1987-04-09 00:00:00
Data variables:
    pr       (time, lat, lon) float32 dask.array<chunksize=(1, 1800, 3600), meta=np.ndarray>
Attributes:
    history:       Created on 2021-12-27 11:47
    CF_variables:  ['precipitation']
    frequency:     day

In [9]:
preproc.ds

<xarray.Dataset>
Dimensions:        (lon: 3600, lat: 1800, time: 792)
Coordinates:
  * lon            (lon) float32 -179.9 -179.8 -179.8 ... 179.8 179.9 179.9
  * lat            (lat) float32 89.95 89.85 89.75 ... -89.75 -89.85 -89.95
  * time           (time) object 1987-01-01 00:00:00 ... 1987-04-09 21:00:00
Data variables:
    precipitation  (time, lat, lon) float32 dask.array<chunksize=(1, 1800, 3600), meta=np.ndarray>
Attributes:
    history:       Created on 2021-12-27 11:47
    CF_variables:  ['precipitation']

Now, use the above object to compute the climate index `DD`.

In [10]:
proc = xcalc.processing(
    "DD", project="MSWEP", crop_time_axis=False, preproc_obj=preproc
)
proc.proc

<xarray.Dataset>
Dimensions:    (lon: 3600, lat: 1800, time: 1, bnds: 2)
Coordinates:
  * lon        (lon) float32 -179.9 -179.8 -179.8 -179.6 ... 179.8 179.9 179.9
  * lat        (lat) float32 89.95 89.85 89.75 89.65 ... -89.75 -89.85 -89.95
  * time       (time) object 1987-07-02 00:00:00
Dimensions without coordinates: bnds
Data variables:
    DD         (time, lat, lon) float64 dask.array<chunksize=(1, 1800, 3600), meta=np.ndarray>
    time_bnds  (time, bnds) object 1986-12-31 12:00:00 1988-12-31 12:00:00
Attributes:
    history:       Created on 2021-12-27 11:47
    CF_variables:  ['precipitation']
    frequency:     day

Finally, do some postprocessing. The postprocessor writes some additional attributes to `ds`' global attributes.

In [11]:
postproc = xcalc.postprocessing(
    project="MSWEP",
    proc_obj=proc,
    institution="Helmholtz-Zentrum hereon GmbH, Climate Service Center Germany",
    institution_id="GERICS",
)
postproc.postproc

[<xarray.Dataset>
 Dimensions:    (lon: 3600, lat: 1800, time: 1, bnds: 2)
 Coordinates:
   * lon        (lon) float32 -179.9 -179.8 -179.8 -179.6 ... 179.8 179.9 179.9
   * lat        (lat) float32 89.95 89.85 89.75 89.65 ... -89.75 -89.85 -89.95
   * time       (time) object 1987-07-02 00:00:00
 Dimensions without coordinates: bnds
 Data variables:
     DD         (time, lat, lon) float64 dask.array<chunksize=(1, 1800, 3600), meta=np.ndarray>
     time_bnds  (time, bnds) object 1986-12-31 12:00:00 1988-12-31 12:00:00
 Attributes: (12/14)
     history:               Created on 2021-12-27 11:47
     CF_variables:          ['precipitation']
     frequency:             day
     ci_creation_date:      2023-10-05T11:08:07Z
     ci_institute_id:       GERICS
     ci_institution:        Helmholtz-Zentrum hereon GmbH, Climate Service Cen...
     ...                    ...
     ci_reference_period:   1971-2000
     ci_frequency:          year
     ci_timerange_source:   19870101-19870409
     

Optionaly, we can store the dataset as a netCDF file on disk. The output file naming depends on the user-given project. 

In [ ]:
xcalc.outputwriter(postproc_obj=postproc)

For simplyfication, we can do preprocessing, processing, postprocessing and output file storing by calling just one function.

In [ ]:
idx_obj = xcalc.index_calculator(
    ds=ds,
    index="DD",
    project="MSWEP",
    institution_id="GERICS",
    institution="Helmholtz-Zentrum hereon GmbH, Climate Service Center Germany",
    write=True,
)

In [ ]:
idx_obj.postproc